# HearHere
## Measure local reputation of global business

### 1. Choose company and extract locations

#### 1.1 'BBVA' offices extractor

In [1]:
import pandas as pd
from config import data_path

In [2]:
#Setup functions requests and path
update_bbva_offices = False
bbva_offices_path = data_path + 'bbva_offices.csv'

#Load data
if update_bbva_offices: 
    
    #Import BBVA offices scraper
    from scrapers import bbva_extract_offices
    
    #Export to .csv
    bbva_offices = bbva_extract_offices()
    bbva_offices.to_csv(bbva_offices_path, index=False)
    
else:
    bbva_offices = pd.read_csv(bbva_offices_path)
    
bbva_offices.head()

,office_number,office_url,office_direction,municipality_name,municipality_url,province_name,province_url
0,92.0,https://www.bbva.es/general/oficinas/almeria/a...,"Adra - Natalio Rivas, 23, C.P:4770, Adra, Alm...",Adra,https://www.bbva.es/general/oficinas/almeria/adra,Almería,https://www.bbva.es/general/oficinas/almeria
1,5764.0,https://www.bbva.es/general/oficinas/almeria/a...,"Aguadulce - Av. Carlos Iii Esq. C/lima, C.P:47...",Aguadulce,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
2,5461.0,https://www.bbva.es/general/oficinas/almeria/a...,"Almeria - Empresas - Paseo Almeria, 41, C.P:40...",Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
3,1809.0,https://www.bbva.es/general/oficinas/almeria/a...,"Almeria - Ps. Almeria 41 - Ps. Almeria, 41, C....",Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
4,6828.0,https://www.bbva.es/general/oficinas/almeria/a...,Almeria - Edf Oliveros - Canonigo Molina Alons...,Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria


#### 1.2 Clean DataFrame

In [3]:
import numpy as np

#Drop rows without office information
bbva_offices.replace('', np.nan, inplace=True)
bbva_offices = bbva_offices.dropna()

bbva_offices.head()

,office_number,office_url,office_direction,municipality_name,municipality_url,province_name,province_url
0,92.0,https://www.bbva.es/general/oficinas/almeria/a...,"Adra - Natalio Rivas, 23, C.P:4770, Adra, Alm...",Adra,https://www.bbva.es/general/oficinas/almeria/adra,Almería,https://www.bbva.es/general/oficinas/almeria
1,5764.0,https://www.bbva.es/general/oficinas/almeria/a...,"Aguadulce - Av. Carlos Iii Esq. C/lima, C.P:47...",Aguadulce,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
2,5461.0,https://www.bbva.es/general/oficinas/almeria/a...,"Almeria - Empresas - Paseo Almeria, 41, C.P:40...",Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
3,1809.0,https://www.bbva.es/general/oficinas/almeria/a...,"Almeria - Ps. Almeria 41 - Ps. Almeria, 41, C....",Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria
4,6828.0,https://www.bbva.es/general/oficinas/almeria/a...,Almeria - Edf Oliveros - Canonigo Molina Alons...,Almeria,https://www.bbva.es/general/oficinas/almeria/a...,Almería,https://www.bbva.es/general/oficinas/almeria


In [7]:
direction = bbva_offices['office_direction'].str.split("-", n=-1, expand=True)

direction

,0,1,2,3,4,5
0,Adra,"Natalio Rivas, 23, C.P:4770, Adra, Almería",None,None,None,None
1,Aguadulce,"Av. Carlos Iii Esq. C/lima, C.P:4720, Aguadul...",None,None,None,None
2,Almeria,Empresas,"Paseo Almeria, 41, C.P:4001, Almeria, Almería",None,None,None
3,Almeria,Ps. Almeria 41,"Ps. Almeria, 41, C.P:4001, Almeria, Almería",None,None,None
4,Almeria,Edf Oliveros,"Canonigo Molina Alonso, 6, C.P:4004, Almeria,...",None,None,None
...,...,...,...,...,...,...
2955,Torrepacheco,"Av. De Fontes, 2, C.P:30700, Torre","Pacheco, Murcia",None,None,None
2956,Las Torres De Cotillas,"Mayor, 26, C.P:30565, Torres De Cotillas (las...",None,None,None,None
2957,Totana,"A. Gomez Pellicer, 1, C.P:30850, Totana, Murcia",None,None,None,None
2958,La Union,"Mayor, 92, C.P:30360, Union (la), Murcia",None,None,None,None
